Getting the probability of the IBP matrix draws took me a while.  I tried a few ways.  Some of which are in the first few paragraphs below.  If you want to skip all the stuff that didnt work, you can just skip down to the paragraph starting "Finally I thought" (I guess you can use ctrl-F Find to get to that easily).  Next few paragraphs are my ramblings (some of them...) whilst trying to find a way forward.

Let's think this through a bit.  If we draw a random matrix, without any priors, then there are $2^{N K^+}$ possible arrangements of 1s and 0s, given we know $K^+$.

But in fact when we draw random matrix, with the infinite prior, there is no restriction on $K^+$, and we need to consider the probability of a matrix compared with any possible matrix drawn, including the ones where $K^+$ is really large, albeit with very tiny probability.

Comparing with the earlier equation for the infinite limit of $P(\mathbf{Z})$, it looks like the tutorial is asserting that:

$$
P(\mathbf{Z})_{IBP} =
\frac
 {\prod_{h=1}^{2^N-1}K_h!}
  {\prod_{i=1}^N K_1^{(i)}!}
P([\mathbf{Z}])
$$

Let's denote the original $P(\mathbf{Z})$, the one drawn from binomial features, conditionally independent on $\pi_k$, as $P(\mathbf{Z})_{BF}$.  And the probability of matrices drawn using the Indian Buffer Process, is $P(\mathbf{Z})_{IBP}$.  So, we have:

$$
P(\mathbf{Z})_{IBP} =
\frac
 {\prod_{h=1}^{2^N-1}K_h!}
  {\prod_{i=1}^N K_1^{(i)}!}
P([\mathbf{Z}])_{BF}
$$

From earlier, we have:

$$P([\mathbf{Z}])_{BF} = \frac{K!}{\prod_{h=0}^{2^{N}-1}K_h!} P(\mathbf{Z})_{BF}$$

So, this means that:

$$
P(\mathbf{Z})_{IBP} =
\frac{K!}
 {K_0!\prod_{i=1}^N K_1^{(i)}!}
P(\mathbf{Z})_{BF}
$$

... but I cant quite see how this arises.  Perhaps, the result that the tutorial gives for $P(\mathbf{Z})_{IBP}$ is derived from first principles?  Let's try that.

For the first customer, we draw Poisson$(\alpha)$ features.  Using the $K_1^{(i)}$ notation, the probability of the first draw is, from Poisson distribution:

$$
\frac{\alpha^{K_1^1}\exp(-\alpha)}{K_1^{(1)}!}
$$

For the second customer, we have the probability of drawing 1 and 0 for each of the existing dishes, and then the probability of drawing the next new dishes.  The number of existing dishes is: $K_1^{(1)}$.  And the probability of drawing... well...we need to know which features customer drew draw 1 for, and which ones they drew 0 for.  And looking at the final expression, we only have $K_1^{(i)}$ and $m_k$ to work with.  But we can calculate the draw for each feature using $m_k$.  Using $z_{2,k}$ for the draw of each feature for customer 2 we have:

$$z_{2,k} = m_{\dots} \dots$$

.. .wait, that wont work, because what we'd really need is the partial history of each feature.  But, what about, we simply calculate the probability of the history of each feature, after the time it's been drawn?  For this we just need to know:

- when that feature was first drawn, so we know how long the history is, and
- how many 1s in that history, which is $m_k$

For the features drawn by customer 1, the length of the history, excluding the first 1, is $N - 1$.  And there are exactly $m_k - 1$ 1s in the history after the first customer, and exactly $N - m_k$ 0s.  So the probability of drawing the rest of the history, given the first customer drew a 1, i, binomial distribution.  We need some notation for the feature vector following customer 1, but we can just use the full history vector, conditional on customer 1 drawing a 1.  And let's use $\mathbf{z}_{*,k}$ to denote the full history of feature k.  Then we have:

$$
P(\mathbf{z}_{*,k} \mid z_{1,k} = 1)
= \mathrm{Binomial}(\mathbf{z}_{*,k}; \dots)
$$

... hmmm.... Still wont work, because the probability in the Binomial distribution is not fixed, eg $\alpha$, but changes over time, ie $\frac{m_{k_{i-1}} }{ i }$.

... but it seems like there's not really any other way of figuring out the "probability of any particular matrix being produced by this process", so let's continue.  Let's introduce some new notation: $m_{i,k}$ is the number of times feature $k$ has been chosen by customers up to, and including, customer $i$.  Looking at the probability of the choice of dishes by customer 2, that were already chosen by customer 1, first let's write down the probability of choosing each dish.  It is:

$$
\frac
  {m_{i-1,k}}
  {i}
$$
$$
= \frac
  {m_{1,k}}
  {2}
$$

The probability of the feature vector for customer 2, for $k \le K_1^{(1)}$, is

$$
P(\mathbf{z}_{2,k \le K_1^{(1)}}) =
\prod_{j=1}^{K_1^{(1)}} 
  \left(
    \frac
      {m_{1,k}}
      {2}
   \right)^{z_{2,j}}
  \left(
    1 - 
    \frac
      {m_{1,k}}
      {2}
   \right)^{1 - z_{2,j}}
$$

Or, for all customers, just the probability of each customer sampling existing dishes, not including the Poisson terms yet, the probability is:

$$
P(\mathbf{z}_{*,k \le K_{1}^{(i-1)}}) =
\prod_{i=2}^N
\prod_{p=1}^{i-1}
\prod_{j=1}^{K_1^{p}} 
  \left(
    \frac
      {m_{i-1,j}}
      {i}
   \right)^{z_{i,j}}
  \left(
    1 - 
    \frac
      {m_{i - 1,j}}
      {i}
   \right)^{1 - z_{i,j}}
$$

Note that there is no prior to add for any of the parameters.  In the original formulation, we were sampling from a Bernoulli distribution, parameterized on $\pi_k$, which had a prior with hyperparameter $\alpha$.  But here we are directly sampling based on $\alpha$, and don't need to integerate over $\pi_k$.  So this is the complete formulation for the probability of a matrix.  I think/hope.  Except, we do need to add in the Poisson draw probabilities.  The Poisson draw probability for customer $i$ is:

$$
\mathrm{Poisson}\left(
  K_1^{(i)}; \frac{\alpha}{i}
\right) =
\frac{(\alpha/i)^{K_1^{(i)}}\exp(-\alpha/i)}{K_1^{(i)}!}
$$

So the full probability is:

$$
P(\mathbf{Z})_{\mathrm{IBP}} =
\prod_{i=1}^N 
  \frac{(\alpha/i)^{K_1^{(i)}}\exp(-\alpha/i)}{K_1^{(i)}!}
\cdot
\prod_{i=2}^N
\prod_{p=1}^{i-1}
\prod_{j=1}^{K_1^{p}} 
  \left(
    \frac
      {m_{j-1,j}}
      {i}
   \right)^{z_{i,j}}
  \left(
    1 - 
    \frac
      {m_{j - 1,j}}
      {i}
   \right)^{1 - z_{i,j}}
$$

Looking at the left hand term, ie:

$$
\prod_{i=1}^N 
  \frac{(\alpha/i)^{K_1^{(i)}}\exp(-\alpha/i)}{K_1^{(i)}!}
$$


... we can factorize this out, since it's just products of things, and also bearing in mind that:

$$
\left( \frac{a}{b} \right)^n = \frac{a^n}{b^n}
$$

into:

$$
\prod_{i=1}^N \alpha^{K_1^{(i)}}
\cdot
\prod_{i=1}^N \frac{1}
             {i^{K_1^{(i)}}}
\cdot
\prod_{i=1}^N \exp(-\alpha/i)
\cdot
\prod_{i=1}^N
  \frac{1}
     {K_1^{(i)}!}
$$

Going through each of these terms in turn:

$$
\prod_{i=1}^N \alpha^{K_1^{(i)}}
$$
$$
=\alpha^{\sum_{i=1}^N K_1^{(i)}}
$$
$$
=\alpha^{K^+}
$$

This term figures in the expression in the tutorial, so this seem like a good start.  Let's look at the next term:

$$
\prod_{i=1}^N \frac{1}
             {i^{K_1^{(i)}}}
$$

Not sure, but the fourth term can be combined with the first term to give:

$$
\frac{\alpha^{K^+}}
  {\prod_{i=1}^N K_1^{(i)}!}
$$

... which is the first term in the tutorial expression.

The third term, ie:

$$
\prod_{i=1}^N \exp(-\alpha/i)
$$

Can be written as:
$$
\exp(-\sum_{i=1}^N \alpha/i)
$$
$$
=\exp \left(-\alpha\sum_{i=1}^N \frac{1}{i} \right)
$$
... which is the second term in the tutorial expression.  So, currently we have:

$$
\frac{\alpha^{K^+}}
  {\prod_{i=1}^N K_1^{(i)}!}
\cdot
\exp \left(-\alpha\sum_{i=1}^N \frac{1}{i} \right)
\cdot
\prod_{i=1}^N \frac{1}
             {i^{K_1^{(i)}}}
\cdot
\prod_{i=2}^N
\prod_{p=1}^{i-1}
\prod_{j=1}^{K_1^{p}} 
  \left(
    \frac
      {m_{i-1,j}}
      {i}
   \right)^{z_{i,j}}
  \left(
    1 - 
    \frac
      {m_{i - 1,j}}
      {i}
   \right)^{1 - z_{i,j}}
$$
Somehow, these last two terms should combine presumably into a product over $k$ of factorial terms.

Let's look at the second part of the expression for the probability of the draws from the IBP:

$$
\prod_{i=2}^N
\prod_{p=1}^{i-1}
\prod_{j=1}^{K_1^{p}} 
  \left(
    \frac
      {m_{i-1,j}}
      {i}
   \right)^{z_{i,j}}
  \left(
    1 - 
    \frac
      {m_{i - 1,j}}
      {i}
   \right)^{1 - z_{i,j}}
$$

We should be able to combine the individual products over features, into something that uses $m_k$ or similar.  $m_k$ is over an entire feature, for all $1 \le i \le N$, though, whereas currently the expression is over all $k$, for specific values of $i$.  So we somehow need to rotate/pivot this product over products of products.

At this point, I tried various ways to pivot these terms, or to refactorize them over $k$, but couldnt seem to find a way.

Finally, I thought: since we need everything factorized over $k$ essentially, let's start from that approach?  We'll have a Poisson draw for each customer, of new features, but for each of the features chosen at a certain depth, $i$, we can try to calculate a probability just for one of those individual features, given the fact that it was first added at depth $i_{k}$, that there are $N$ customers in total, the $m_k$ total, and $\alpha$.

If feature $k$ was chosen at depth $i_k$, that means:

- number of 1s, excluding the first 1, is $m_k - 1$
- number of 0s, after the first 1, is $N - i_k - m_k + 1$

(check, eg:

- 5 customers
- feature chosen first by customer 3
- $m_k$ = 2, let's say
- then number of 1s, after customer 3, = $m_k - 1$ = 1
- number of 0s after being chosen = 1 = $5 - 3 - 2 + 1$

)

The probability of this feature $k$, given it was chosen by customer at depth $i_k$ first will be:

$$
\prod_{i=i_k + 1}^N
  \left(
    \frac
      {m_{i - 1,k}}
      {i}
   \right)^{z_{i,k}}
  \left(
    1 - 
    \frac
      {m_{i - 1,k}}
      {i}
   \right)^{1 - z_{i,k}}
$$
...where $m_{i-1,k}$ is the number of times the feature was chosen, up to and including customer $i-1$.  Let's take out the factor of $1/i$.  Since the factor is applied for all features in the loop, and exactly one of the two terms is non-1 for each value of $i$, so it simply becomes, outside the loop:

$$
=
\prod_{i=i_k + 1}^N
  \frac{1}
    {i}
\cdot
\prod_{i=i_k + 1}^N
  \left(
      m_{i - 1,k}
   \right)^{z_{i,k}}
  \left(
    i - 
      m_{i - 1,k}
   \right)^{1 - z_{i,k}}
$$

$$
=
  \frac{1}
    {\prod_{i=i_k + 1}^Ni}
\cdot
\prod_{i=i_k + 1}^N
  \left(
      m_{i - 1,k}
   \right)^{z_{i,k}}
  \left(
    i - 
      m_{i - 1,k}
   \right)^{1 - z_{i,k}}
$$

$$
=
  \frac{i_k!}
    {N!}
\cdot
\prod_{i=i_k + 1}^N
  \left(
      m_{i - 1,k}
   \right)^{z_{i,k}}
  \left(
    i - 
      m_{i - 1,k}
   \right)^{1 - z_{i,k}}
$$

Let's try factorizing out the two products:

$$
=
  \frac{i_k!}
    {N!}
\cdot
\prod_{i=i_k + 1}^N
  \left(
      m_{i - 1,k}
   \right)^{z_{i,k}}
\cdot
\prod_{i=i_k + 1}^N
  \left(
    i - 
      m_{i - 1,k}
   \right)^{1 - z_{i,k}}
$$

For the middle term, although the $m$ and $z$ terms contain $i$, note that:

- there will be exactly $m_k - 1$ occurrences where $z$ is 1
- the $m$ values will follow the sequence $1, \dots, m_k - 1$

Therefore the middle term is:

$$
\prod_{i=1}^{m_k - 1} i
$$

$$
=(m_k - 1)!
$$

This matches the tutorial :-)  Looking at the tutorial expression, the other term will be $(N - m_k)$.  It seems like it's basically the same, but the other way around?  Maybe we can use the same approach as for the first term, and use a substitution so we "count backwards"?  Let's try substituting $s = N + 1 - i$, so $i - 1 = N - s$:

$$
\prod_{i=i_k + 1}^N
  \left(
    i - 
      m_{i - 1,k}
   \right)^{1 - z_{i,k}}
$$

$$
=
\prod_{s=N + 1 - i_k - 1}^{(N + 1 - N)}
  \left(
    N - s + 1 - 
      m_{N - s,k}
   \right)^{1 - z_{N - s,k}}
$$

Swapping the limits around:
$$
=
\prod_{s=1}^{N - i_k}
  \left(
    N - s + 1 - 
      m_{N - s,k}
   \right)^{1 - z_{N - s,k}}
$$

So, hopefully the $m$ here, parameterized on $s$, will work like the earlier $m$, but in reverse, counting backwards.  So, let's think this through, working backwards from the $N$th customer:

- for customer $i=N$, feature $k$ might be 1 or 0
- $s$ will be $N + 1 - i = N + 1 - N = 1$
- $m_{N  - s,k}$ will be $m_{N - 1,k}$

If feature $N$ is a 1, then:

- $m_{N - 1, k}$ will be $m_k - 1$

Otherwise it will be:

- $m_k$

However if it is a 1, then $z_{N -s ,k}$ will be 1, giving a power of $0$, so the entire term becomes $1$, and can be ignored.  In the case where the last feature is a 0, then the $m$ term, for $s=1$, will be: $m_k$.

We can have an arbitrary number of 1s at the bottom of the feature column, prior to the first 0.  When we get to the first 0, then let's look at the values of some of these variables:

- $z$ will be $0$, so the power of $1-z$ is $1$
- $m_{N - s, k}$ will be

... this still wont work, since we want some value of $m$ that increases contiguously, $1,2,3\dots$.  What we need really is a count of the number of zeros, starting from the end.  Let's denote that as $m'_{i,k}$, where $m'_{i,k}$ means the number of zeros in the customers after, and including customer $i$, for feature $k$.

~~What is the relationship between m'_{i,k} and m_{i,k}?~~

Let's now try to write down the probability of the customers who skipped this feature, in terms of $m'_{i,k}$?  The probability of choosing the feature for customer $i$ is:

$$
\frac
  {m_{i-1,k}}
  {i}
$$

The probability of not choosing it is:

$$
1 - 
\frac
  {m_{i-1,k}}
  {i}
$$

The total number of 1s, excluding the first 1, is $m_k - 1$.  The total number of zeros, after the first customer chose the feature, is $N - i_k - m_k + 1$.  The number of zeros after customer $i$ is the total number of zeros after customer $i_k$ minus the number of zeros up to and including customer $i$.  The number of zeros up to and including customer $i$ is:

$$
i - i_k - m_{i,k} + 1
$$

Check for off-by-one errors, let's imagine:

- 5 customers, so N = 5
- $i_k$ is 3
- $m_k$ is 2, and we have $z$ is 1 for customers 3 and 5
- then the number of zeros up to and including customer 4 is 1, and trying the formula above is: $4 - 3 - m_{4,k} + 1$
- $m_{4,k}$ in this case is: 1
- so we get: $4 - 3 - 1 + 1= 1$ (after tweaking the formula, so this became correct :-P )

Therefore:

$$
m'_{i+1,k} = N - i_k - m_k + 1 - i + i_k + m_{i,k} - 1
$$

$$
= N - m_k - i + m_{i,k}
$$

Let's try substituting this into the right hand term from earlier.

$$
\prod_{i=i_k + 1}^N
  \left(
    i - 
      m_{i - 1,k}
   \right)^{1 - z_{i,k}}
$$

$$
= \prod_{i=i_k+1}^N
(N - m_k - m'_{i+1,k})^{1 - z_{i,k}}
$$

Cool.  Now this term is free of $i$ :-)

So, now, just as for the "forwards"-term, the term that was for the probability of the 1s, we should be able to work backwards, and form a sequence, like $1,2,3,\dots$ for the value of each term in this product.  Let's try that.  So, working backwards from the last customer, the first time there is a zero we will have:

- $m_{i+1,k} = 0$
- $N$ is $N$, $m_k$ is $m_k$, and $z$ will be 0

So, the value at the time of the first zero, working backwards, will be: $N - m_k$

For the next zero, working backwards, we will have:

- $m_{i+1,k} = 1$
- the other values are all hte same as before
- so this value will be: $N - m_k - 1$

So, we will have a product of the terms in the sequence: $N - m_k, N - m_k - 1, N - m_k - 2, \dots$

The last term in the sequence will be for the first zero, after customer $i_k$, and there are $N - m_k - i_k + 1$ zeros, so the sequence will be:

$$N - m_k, N - m_k - 1, N - m_k - 2, \dots, N - m_k - (N - m_k - i_k + 1 - 1)$$

$$
= \prod_{j=0}^{N - m_k - i_k} (N - m_k - j)
$$

$$
= \frac{(N - m_k)!}
  {(N - m_k - N + m_k + i_k - 1)!}
$$

$$
=\frac{(N - m_k)!}
  {(i_k - 1)!}
$$

Ok, so let's combine all the terms, for one specific feature, given that customer $i_k$ first chose it.  We have:

$$
\prod_{i=i_k + 1}^N
  \left(
    \frac
      {m_{i - 1,k}}
      {i}
   \right)^{z_{i,k}}
  \left(
    1 - 
    \frac
      {m_{i - 1,k}}
      {i}
   \right)^{1 - z_{i,k}}
$$


$$
  \frac{i_k!}
    {N!}
\cdot
\prod_{i=i_k + 1}^N
  \left(
      m_{i - 1,k}
   \right)^{z_{i,k}}
\cdot
\prod_{i=i_k + 1}^N
  \left(
    i - 
      m_{i - 1,k}
   \right)^{1 - z_{i,k}}
$$

$$
= \frac{i_k!}{N!}
\cdot
(m_k-1)!
\cdot
\frac{(N-m_k)!}
{(i_k-1)!}
$$
The $i_k$ factorials cancel mostly, giving
$$
\frac{i_k}{N!}
(m_k-1)!
{(N-m_k)!}
$$

This is a pretty close match to what is in the tutorial.  Now, we simply need to multiply out by the Poisson probabilities, which we already looked at doing earlier.

So, what we need to do now is:

- we draw a number of features from the Poisson
- we take the power, or the product, for this number of features, of the per-feature expression just above
- and we repeat for each customer

For the first customer, the draw from the Poisson is:

$$
\frac{\alpha^{K_1^{(1)}}\exp(-\alpha)}{K_1^{(1)}!}
$$

The number of features drawn is $K_1^{(1)}$, so the probability for these features, including hte Poisson probabiliyt, is:

$$
\frac{\alpha^{K_1^{(1)}}\exp(-\alpha)}{K_1^{(1)}!}
\frac{i_k}{N!}
(m_k-1)!
{(N-m_k)!}
$$

And here $i_k$ is $1$, so:

$$
\frac{\alpha^{K_1^{(1)}}\exp(-\alpha)}{K_1^{(1)}!}
\frac{1}{N!}
(m_k-1)!
{(N-m_k)!}
$$

For customer $i$, the Poisson draw is:

$$
\mathrm{Poisson}\left(
  K_1^{(i)}; \frac{\alpha}{i}
\right) =
\frac{(\alpha/i)^{K_1^{(i)}}\exp(-\alpha/i)}{K_1^{(i)}!}
$$

... and the probability of the new features they choose, and the choices of all the customers after them, for those dishes, is:

$$
\frac{(\alpha/i)^{K_1^{(i)}}\exp(-\alpha/i)}{K_1^{(i)}!}
\frac{i}{N!}
(m_k-1)!
{(N-m_k)!}
$$

And the total probability over all customers, the probability of the matrix draw that we want, is:

$$
\prod_{i=1}^N
\frac{(\alpha/i)^{K_1^{(i)}}\exp(-\alpha/i)}{K_1^{(i)}!}
\left(
\frac{i}{N!}
(m_k-1)!
{(N-m_k)!}
\right)^{K_1^{(i)}}
$$


Then, copying and pasting from ramblings earlier, looking at the first term, the product of the Poisson probabilities, we can factorize this out, since it's just products of things, and also bearing in mind that:

$$
\left( \frac{a}{b} \right)^n = \frac{a^n}{b^n}
$$

into:

$$
\prod_{i=1}^N \alpha^{K_1^{(i)}}
\cdot
\prod_{i=1}^N \frac{1}
             {i^{K_1^{(i)}}}
\cdot
\prod_{i=1}^N \exp(-\alpha/i)
\cdot
\prod_{i=1}^N
  \frac{1}
     {K_1^{(i)}!}
$$

Going through each of these terms in turn:

$$
\prod_{i=1}^N \alpha^{K_1^{(i)}}
$$
$$
=\alpha^{\sum_{i=1}^N K_1^{(i)}}
$$
$$
=\alpha^{K^+}
$$

This matches one of the terms in the tutorial expression.  The fourth term can be combined with this term to give:

$$
\frac{\alpha^{K^+}}
  {\prod_{i=1}^N K_1^{(i)}!}
$$

... which matches the first term in the tutorial expression.

The third term, ie:

$$
\prod_{i=1}^N \exp(-\alpha/i)
$$

Can be written as:
$$
\exp(-\sum_{i=1}^N \alpha/i)
$$
$$
=\exp \left(-\alpha\sum_{i=1}^N \frac{1}{i} \right)
$$
... which is the second term in the tutorial expression.  So, this gives overall for now:

$$
\frac{\alpha^{K^+}}
  {\prod_{i=1}^N K_1^{(i)}!}
\cdot
\exp \left(-\alpha\sum_{i=1}^N \frac{1}{i} \right)
\cdot
\prod_{i=1}^N \frac{1}
             {i^{K_1^{(i)}}}
\cdot
\prod_{i=1}^N
\left(
\frac{i}{N!}
(m_k-1)!
{(N-m_k)!}
\right)^{K_1^{(i)}}
$$

The $i$s cancel, giving:

$$
\frac{\alpha^{K^+}}
  {\prod_{i=1}^N K_1^{(i)}!}
\cdot
\exp \left(-\alpha\sum_{i=1}^N \frac{1}{i} \right)
\cdot
\prod_{i=1}^N
\left(
\frac{1}{N!}
(m_k-1)!
{(N-m_k)!}
\right)^{K_1^{(i)}}
$$

So close, and yet :-P